In [1]:
from galois import GF, lagrange_poly, Poly
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize
import numpy as np

  

# The L, R, and O matrices from the R1CS example
# p = 239
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = GF(p)

x = FP(2)
y = FP(3)
  


L = FP([

[0, 1, 0, 0, 0, 0, 0, 0],

[0, 0, 1, 0, 0, 0, 0, 0],

[0, 0, 0, 0, 4, 0, 0, 0],

[0, 13, 0, 0, 0, 0, 0, 0],

[0, 0, 0, 5, 0, 0, 0, 0],

])

  

R = FP([

[0, 1, 0, 0, 0, 0, 0, 0],

[0, 0, 1, 0, 0, 0, 0, 0],

[0, 0, 0, 1, 0, 0, 0, 0],

[0, 0, 0, 0, 1, 0, 0, 0],

[0, 1, 0, 0, 0, 0, 0, 0]

])

  

O = FP([

[0, 0, 0, 1, 0, 0, 0, 0],

[0, 0, 0, 0, 1, 0, 0, 0],

[0, 0, 0, 0, 0, 1, 0, 0],

[0, 0, 0, 0, 0, 0, 1, 0],

[0, 0, 10, FP(p - 1), 0, 1, FP(p - 1), 1]

])

w = FP([1,2,3,4,9,144,234,104,])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)


print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


Lw = [ 2  3 36 26 20]
Rw = [2 3 4 9 2]
Lw * Rw = [  4   9 144 234  40]
Ow =      [  4   9 144 234  40]


In [2]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''L
{Lp}
''')

print(f'''R
{Rp}
''')

print(f'''O
{Op}
''')

L
[[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [21888242871839275222246405745257275088548364400416034343698204186575808495557
   5472060717959818805561601436314318772137091100104008585924551046643952124030
  19152212512859365819465605027100115702479818850364030050735928663253832433579
  16416182153879456416684804308942956316411273300312025757773653139931856371736
   2736030358979909402780800718157159386068545550052004292962275523321976061950]
 [21888242871839275222246405745257275088548364400416034343698204186575808495607
   3648040478639879203707734290876212514758060733402672390616367364429301415954
  1824020239319939601853867145438106

In [3]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
tau = FP(20)

print(f"τ = {tau}")

T = Poly([1, p-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= Poly([1, p-i], field=FP)

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

Setup phase
----------
Toxic waste:
τ = 20

T =  x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
----------
T(6) =  120

T(τ) = 1395360


In [4]:
# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)]
[T(τ)]G1 = [(1641247283492879903468444805169804215277964208279225379119694118848884481979, 18733245328562972535068072505811612803902036184072974454572123739995203670419), (1282524939337382483469274134339301655621870987447481453798307725808867601190, 155391820121621028156365747107525575876406255471822024656389149920860

In [5]:
U = Poly((w @ Lp)[::-1])
V = Poly((w @ Rp)[::-1])
W = Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}"

U =  20064222632519335620392538599819168831169334033714698148390020504361157787654x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831809x^3 + 1824020239319939601853867145438106257379030366701336195308183682214650708237x^2 + 14592161914559516814830937163504850059032242933610689562465469457717205663306x + 230
V =  18240202393199396018538671454381062573790303667013361953081836822146507079680x^4 + 9x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301415903x^2 + 50x + 21888242871839275222246405745257275088548364400416034343698204186575808495594
W =  14592161914559516814830937163504850059032242933610689562465469457717205663742x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247806x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602832019x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247431x + 240
H =  51680573447398288719192902454079677292405860389871192200398

In [6]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    if verbose:
        [print(normalize(point)) for point in trusted_points]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    if verbose:
        print("-"*10)
        print(normalize(evaluation))
    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
B_G1 = evaluate_poly(V, tau_G1)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")


print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")


Proof generation:
----------
[A]G1 = (12974593486416814985020255880197160193274347462627434780332815615415470171229, 13418989303959161452717776245004303140000382733472958116551618870616574929886)
[B]G2 = ((12803249360509112016217557357634139462067373039035579739317806663976097101968, 6342654981983416857189382116288306537222945437003598845620250344655630825711), (8791369942080430333533513748393454786849947027282241721228523155320183432197, 16662234193013290094789917963688928599183871668161591655609772976921414776267))
[C]G1 = (12529780198059822267895103632117873348094767660290233868231973319478637008304, 628172615926452590176647916841804396725003774477048543561085306153859680182)

Proof verification:
----------
Pairing check passed!


## Introducting Alpha & Beta into the inputs to ensure that a forged proof cannot be submitted & verified

This will transform the equation to evaluate into the following:

```bash
(A + alpha)(B + beta) = (alpha * beta) + (beta * A) + (alpha * B) + C
```

In [9]:
## Adjust our trusted setup so that it now supplies values for alpha and beta
def evaluate_poly_list(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"τ = {tau}")



Setup phase
----------
Toxic waste:
α = 2
β = 3
τ = 20


In [12]:
## Evaluate the preimage of (beta * A) + (alpha * B) + C

beta_L = beta * Lp
alpha_R = alpha * Rp
K = beta_L + alpha_R + Op # preimage of [βA + αB + C]

Kp = to_poly(K)
print_poly("K", Kp)

print("K evaluations:")
## Evaluate all of the polynomials in the matrix using the value of tau
K_eval = evaluate_poly_list(Kp, tau)
print([int(k) for k in K_eval])


K polynomials:
K_0 = 0
K_1 = 4560050598299849004634667863595265643447575916753340488270459205536626769914x^4 + 5472060717959818805561601436314318772137091100104008585924551046643952123972x^3 + 17328192273539426217611737881662009445100788483662693855427744981039181725448x^2 + 16416182153879456416684804308942956316411273300312025757773653139931856372073x + 21888242871839275222246405745257275088548364400416034343698204186575808495449
K_2 = 9120101196599698009269335727190531286895151833506680976540918411073253539840x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831879x^3 + 12768141675239577212977070018066743801653212566909353367157285775502554955742x^2 + 14592161914559516814830937163504850059032242933610689562465469457717205663813x + 21888242871839275222246405745257275088548364400416034343698204186575808495577
K_3 = 19152212512859365819465605027100115702479818850364030050735928663253832433666x^4 + 912010119659969800926933572719053128689515183350668097654091841

In [20]:
## Encrypted evaluation step by the setup agent: 
# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd(τ) + αVd(τ) + Wd(τ)]
k_G1 = [multiply(G1, int(k)) for k in K_eval]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[k]G1 = {[normalize(point) for point in k_G1]}")
print(f"Length of encrypted k_eval: {len(k_G1)} \nLength of non-encrpted k_eval : {len(K_eval)}")

print(f"[τT(τ)]G1 = {[normalize(point) for point in target_G1]}")
print(f"\n[β]G2 = {normalize(beta_G2)}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")


Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441416830039777911637913418824951667761761)
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843

In [25]:
## Now our prover needs to incorporate the encrpyted values of alpha and beta into their proof
## to satisfy the new requirements of the verifier as specified by the trusted setup agent

print("\nProof generation:")
print("-"*10)

U = Poly((w @ Lp)[::-1])
V = Poly((w @ Rp)[::-1])

# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)

# G1[A] = G1[A] + G1[α]
A_G1 = add(A_G1, alpha_G1)

# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)

# G2[B] = G2[B] + G2[β]
B_G2 = add(B_G2, beta_G2)

# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)
assert len(w) == len(k_G1), "Polynomial degree mismatch!"

# w0 * G1[k0] + w1 * G1[k1] + ... + wd-1 * G1[kd-1]
K_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_G1, w)]

K_G1 = K_G1_terms[0]
for i in range(1, len(K_G1_terms)):
    K_G1 = add(K_G1, K_G1_terms[i])

C_G1 = add(HT_G1, K_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")
print("-" * 10)
print("Verifier uses:")
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G2)}")


Proof generation:
----------
[A]G1 = (2064430016998063779353798725494295938510033570286085637920979922823529914045, 11163607427677292198086904898742192686118651690403161551689462855076808244786)
[B]G2 = ((9379046325560130406292105118491912585512872816957156094865020637942426737446, 18922197764223081518953110924117923760865927663005267733120456934193918085640), (19287159416571646716477490319551260722876879850544316937354427240244882161840, 3367833720826567747182456585873250679881805484133693446398305445816768906977))
[C]G1 = (11358665535552242883346533284578950659639409549326885601808688368897936146513, 1127134210402406810000753349328307110913674674587121388539844001384791090499)
----------
Verifier uses:
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 72731651027999311117158714715503

From here, we are able to provide these proofs into contract #1 to verify the proofs generated. The values above have been hard coded into the contract for east tests, but a TODO would be to hand them in as variables when calling the contract. Possible rework with Foundry for testing

## Breaking the Witness Vector into Public & Private Components

In [26]:
## Now we need to split the vector into it's private, and public components.
## 1, out are public, while the rest of the inputs and the intermediate values are private.

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return Poly(p1, field=FP), Poly(p2, field=FP)

u = U(tau)
v = V(tau)
_w = W(tau) # w taken by witness vector
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

##  This is the polynomial for the private input. Represents [βU+αV+W+HT]
c = (beta * u2 + alpha * v2 + w2) + ht 
## This is the polynomial for the public input. Represents [βU+αV+W]
k = (beta * u1 + alpha * v1 + w1)

assert (u + alpha) * (v + beta) == alpha * beta + k + c # should be equal

In [28]:
## The prover will supply A′, B′, and C. The values of α and β are provided to the verifier from the trusted setup.
##  The verifier is responsible for calculating the point K. The prover is required to present the values [1 out] 
## in an unencrypted form. To encrypt these values, the verifier will employ elliptic curve scalar multiplication,
## and the necessary points for this operation will be supplied by the setup ([k1] * 1 + [k2] * out). - Crypto Fairy

k_pub_G1, k_priv_G1 = k_G1[:2], k_G1[2:]
pub_input, priv_input = w[:2], w[2:]

print(f"[k_pub]G1 = {[normalize(point) for point in k_pub_G1]}")
print(f"[k_priv]G1 = {[normalize(point) for point in k_priv_G1]}")

print(f"pub_input = {pub_input}")
print(f"priv_input = {priv_input}")

[k_pub]G1 = [(0, 0), (13528591474082492817989364351907050604558391337621820406021384540557209749643, 14443303276765745692271933246133099141967259652638964362265269266367251910074)]
[k_priv]G1 = [(9255362000568086969426581910999695218767488354011423148343055923150063610530, 19663255005511524857343047157430517662352787346134121583487236661223935168532), (18924472558501724620782249633268437214267522045596653136491458453906112200618, 8360864883368290615591978878258384068158407909561814807518538073226891889674), (14178798183015274704957343565552930561463683609369585424184893028960793179429, 18179400826732874310473614542147253881358114838912212848392105681367400105891), (21873831532653989422124438111341799219720302685523682132154663052949407849524, 21483527558377832820635913287149611726044468949019950459613875955482194096797), (21440261908337528660304410226561577129474777636665770612314346500965861946682, 349989637048099952648310051560239728796731057825447659609683305681324513939), (17518925

In [32]:
## Now the prover will calculate his portion of the proof:
K_priv_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_priv_G1, priv_input)]
K_priv_G1 = K_priv_G1_terms[0]
for i in range(1, len(K_priv_G1_terms)):
    K_priv_G1 = add(K_priv_G1, K_priv_G1_terms[i])

C_G1 = add(HT_G1, K_priv_G1)


normalize(C_G1)

(4696614388732208573829558575249298593120327644460963806380152762064195939315,
 5640009225556751408734476549072862475628386315255489818368462316990052828449)

From here, we are able to provide these proofs into contract #2 to verify the proofs generated. The values above have been hard coded into the contract for east tests, but a TODO would be to hand them in as variables when calling the contract. Possible rework with Foundry for testing

## Introducing γ and δ

Now in order to ensure the integrity of the public portion of the proof, we must introduce two more variables into the trusted setup: γ and δ.

This will alter the K and C portion of the A'B' == (alpha * beta) + K + C equation

In [35]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4) # <-- new
delta = FP(5) # <-- new
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return Poly(p1, field=FP), Poly(p2, field=FP)

u = U(tau)
v = V(tau)
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

# Essentially dividing both sides by gamma and delta:
# private input divided by delta
c = (beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1
# public input divided by gamma
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

a = u + alpha
b = v + beta

assert a * b == alpha * beta + k * gamma + c * delta # should be equal.

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20


In [37]:
# New Trusted setup values

print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]
delta_G2 = multiply(G2, int(delta))
gamma_G2 = multiply(G2, int(gamma))

powers_tauTtau_div_delta = [(tau**i * T_tau) / delta for i in range(0, T.degree - 1)]
target_G1 = [multiply(G1, int(pTd)) for pTd in powers_tauTtau_div_delta]

assert len(target_G1) == len(H.coefficients()), f"target_G1 length mismatch! {len(target_G1)} != {len(H.coefficients())}"

K_gamma, K_delta = [k/gamma for k in K_eval[:2]], [k/delta for k in K_eval[2:]]
K_gamma_G1 = [multiply(G1, int(k)) for k in K_gamma]
K_delta_G1 = [multiply(G1, int(k)) for k in K_delta]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G2 = {normalize(beta_G2)}")
print(f"[γ]G2 = {normalize(gamma_G2)}")
print(f"[δ]G2 = {normalize(delta_G2)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")
print(f"[τT(τ)/δ]G1 = {[normalize(point) for point in target_G1]}")
print(f"[K/γ]G1 = {[normalize(point) for point in K_gamma_G1]}")
print(f"[K/δ]G1 = {[normalize(point) for point in K_delta_G1]}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20
Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
[γ]G2 = ((18936818173480011669507163011118288089468827259971823710084038754632518263340, 18556147586753789634670778212244811446448229326945855846642767021074501673839), (18825831177813899069786213865729385895767511805925522466244528695074736584695, 13775476761357503446238925910346030822904460488609979964814810757616608848118))
[δ]G2 = ((20954117799226682825035885491234530437475518021362091509513177301640194298072, 

In [47]:
# Now, the prover updates their portion of the proof to include the new values of gamma and delta:
A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
HT_G1 = evaluate_poly(H, target_G1)

# [K/δ*w_priv]G1
Kw_delta_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_delta_G1, priv_input)]
Kw_delta_G1 = Kw_delta_G1_terms[0]
for i in range(1, len(Kw_delta_G1_terms)):
    Kw_delta_G1 = add(Kw_delta_G1, Kw_delta_G1_terms[i])

C_G1 = add(Kw_delta_G1, HT_G1)


print("Prover Portion:")
print("-"*10)
print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")
print("\n")
print("Verifier Portion:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G2 = {normalize(beta_G2)}")
print(f"[γ]G2 = {normalize(gamma_G2)}")
print(f"[δ]G2 = {normalize(delta_G2)}")

print(f"[K/γ]G1 = {[normalize(point) for point in K_gamma_G1]}")

Prover Portion:
----------
[A]G1 = (2064430016998063779353798725494295938510033570286085637920979922823529914045, 11163607427677292198086904898742192686118651690403161551689462855076808244786)
[B]G2 = ((9379046325560130406292105118491912585512872816957156094865020637942426737446, 18922197764223081518953110924117923760865927663005267733120456934193918085640), (19287159416571646716477490319551260722876879850544316937354427240244882161840, 3367833720826567747182456585873250679881805484133693446398305445816768906977))
[C]G1 = (16368541905703294401631609256217731528347762741003842614083186838230918302855, 2671812455167606325095718527386883499207019764175431232692428774115388234056)


Verifier Portion:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 727316510279993111171587147155

These values have been hard coded into contract #3 to verify the proof we have generated thus far. Please refer to that contract to see the verification in action as well as how all these values are structured and used in the context of a smart contract. 

## Last step: r & s 

To ensure the integrity of the prover's proof, he must introduce randomness in its generation. This is done to guarantee that every generated proof is unique. 

To accomplish this, the prover must generate random points, r and s, each time the proof is generated. 

The verifier must keep track of the "used" or submitted proofs. In this step, the verifier's porrtion remains unchanged, but the prover's portion must be augmented to include r and s.

In [48]:
print("Prover picks random r, s")
r = FP(12)
s = FP(13)

print(f"r = {r}")
print(f"s = {s}")

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return Poly(p1, field=FP), Poly(p2, field=FP)

u = U(tau)
v = V(tau)
ht = H(tau)*T_tau

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

a = u + alpha + r * delta
b = v + beta + s * delta

c = ((beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1) + s * a + r * b - r * s * delta
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

assert a * b == alpha * beta + k * gamma + c * delta # should be equal

Prover picks random r, s
r = 12
s = 13


In [51]:
# Now to encrypt the new prover proofs:
delta_G1 = multiply(G1, int(delta))
r_delta_G1 = multiply(delta_G1, int(r))
s_delta_G1 = multiply(delta_G1, int(s))
s_delta_G2 = multiply(delta_G2, int(s))

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
A_G1 = add(A_G1, r_delta_G1)

B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
B_G2 = add(B_G2, s_delta_G2)

B_G1 = evaluate_poly(V, tau_G1)
B_G1 = add(B_G1, beta_G1)
B_G1 = add(B_G1, s_delta_G1)


As_G1 = multiply(A_G1, int(s))
Br_G1 = multiply(B_G1, int(r))
rs_delta_G1 = multiply(delta_G1, int(-r*s))

C_G1 = add(Kw_delta_G1, HT_G1)
C_G1 = add(C_G1, As_G1)
C_G1 = add(C_G1, Br_G1)
C_G1 = add(C_G1, rs_delta_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")

[A]G1 = (13497804651742918647529708842475132375367226104443408059072699395631349563480, 2473204928452567718553706936231293669733426625629871455380242433167749458271)
[B]G2 = ((7542190384444271024032144950551386527079129976278292398341289514253421960245, 18125424378557757452068257611299860131247079052606606315921743195924741814406), (4257626515766978681312923731192415907516765455439162554687252584595642898839, 14902578594057989088566240664773415066008599241803712986418508560479626954117))
[C]G1 = (21748123367770491919677225130943491088816972483242991171573332532299299413102, 2117141204141257043815546025964548737968445832358836941175058784210839368886)


With that, Groth16 has been fully implemented. Contract #4 is essentially the same as Contract #3, with the exception that the proofs for A, B, C have been updated to include the randomness provided by the points, r & s. 